# BASIC promoter and CDS genbank files

## Aims and objectives

for relevant promoters and CDSs:
- [x] Generate FASTA sequences.
- [x] Upload to Benchling.
- [x] Make benchling feature libraries and auto-annotate.
- [x] Export annotated genbank files.
- [x] Assemble each into BASIC_SEVA_18, remove color annotations and add meta-data.
- [x] Consolodate parts into a single genbank file in parts_linkers sub-package.
- [x] Generate dict for users.


In [1]:
from Bio import SeqIO
from pathlib import Path

DIR_MISC_BASIC = Path.cwd().parent / "sequences" / "genbank_files" / "misc_BASIC"
promoters = SeqIO.parse(DIR_MISC_BASIC /  "initial_BASIC_promoters.gb", "genbank")
SeqIO.write(promoters, DIR_MISC_BASIC / "initial_BASIC_promoters.fasta", "fasta")

63

## Objectives for cell below

Complete promoters according to the overal aims/objectives.

In [4]:
import basicsynbio as bsb
from basicsynbio.utils import (
    only_label_feature,
    all_feature_values
)
import re


def filter_list_w_re(regular_expression, mylist: list):
    "Filters the list using the given regular expression."
    r = re.compile(regular_expression)
    return filter(r.match, mylist)


def process_promoters(promoters):
    """Returns promoters as required by objectives."""
    for promoter in promoters:
        promoter = only_label_feature(promoter)
        promoter.all_feature_values = all_feature_values(promoter)
        promoter.terminator = next(filter_list_w_re("Terminator.*", promoter.all_feature_values))
        try:
            promoter.promoter = next(filter_list_w_re("J23.*", promoter.all_feature_values))
        except StopIteration:
            for promoter_id in ["Phlf", "CymR", "TetR", "VanR", "LuxR", "CinR", "LacI", "AraC", "BetI", "Ttg", "SaITTC", "T7_100", "T7_52", "T7_50", "T7_25"]:
                if promoter_id in promoter.all_feature_values:
                    promoter.promoter = promoter_id
        promoter.riboj = next(filter_list_w_re("Ribo.*", promoter.all_feature_values))
        promoter_part = bsb.BasicAssembly(
            bsb.BIOLEGIO_LINKERS["LMP"],
            promoter,
            bsb.BIOLEGIO_LINKERS["LMS"],
            bsb.BSEVA_PARTS["18"]
        )
        promoter_part = promoter_part.return_part(
            id=promoter.id,
            name=f"{promoter.id}_{promoter.terminator}_{promoter.promoter}_{promoter.riboj}",
            description=f"{promoter.id} stored in BASIC_SEVA_18"
        )
        yield promoter_part


annotated_promoters = bsb.import_parts(DIR_MISC_BASIC / "annotated_BASIC_promoters.gb", "genbank")
processed_promoters = process_promoters(annotated_promoters)
DIR_PARTS_LINKERS = Path.cwd().parent / "basicsynbio" / "parts_linkers"
bsb.export_to_file(
    processed_promoters,
    DIR_PARTS_LINKERS / "BASIC_promoter_collection.gb"
)




## Objectives for cell below

Complete CDSs for overal aim and objectives

In [8]:
from basicsynbio.utils import _easy_seqrec, feature_from_qualifier


class CDSPart():
    def __init__(self, id, handle_to_parent):
        self.id = id
        self.handle_to_parent = handle_to_parent
        self.part = bsb.import_part(self.handle_to_parent, "genbank")
        self.cds_part = self.make_part()

    def make_part(self):
        try:
            cds_feature = feature_from_qualifier(self.part, "gene", [self.id])
            cds_seq = cds_feature.extract(self.part.seq)
        except KeyError:
            print(f"{self.id} was not found in the corresponding BasicPart.")
        cds_seqrec = _easy_seqrec(
            str(cds_seq),
            self.id,
            annotation_type="CDS",
            label=[self.id]
        )
        return bsb.seqrec2part(cds_seqrec, add_i_seqs=True)

    def stored_part(self, backbone=bsb.BSEVA_PARTS["18"]):
        cds_assembly = bsb.BasicAssembly(
            backbone,
            bsb.BIOLEGIO_LINKERS["LMP"],
            self.cds_part,
            bsb.BIOLEGIO_LINKERS["LMS"]
        )
        stored_part = cds_assembly.return_part(
            self.id,
            description=f"{self.id} stored in {backbone.id}",
            name=f"BASIC_{self.id}_CDS"
        )
        return stored_part


cdss = [
    CDSPart("sfGFP", DIR_MISC_BASIC / "BASIC_sfGFP_ORF.1.gb"),
    CDSPart("mCherry", DIR_MISC_BASIC / "BASIC_mCherry_ORF.1.gb"),
    CDSPart("mTagBFP2", DIR_MISC_BASIC / "BASIC_mTagBFP2_ORF.1.gb")
]
bsb.export_to_file(
    (cds.stored_part() for cds in cdss),
    DIR_PARTS_LINKERS / "BASIC_CDS_collection.gb"
)

